In [ ]:
Picked 20000 records of Semantic Scholar
Targeted record linkage with dblp to get citations and references
Found around ? records on dlbp with a decent score by dblp, top hits with the max score
performed record linkage using title and author matches
got around 13k matches

In [12]:
import os,json
import pandas as pd
import ast

In [7]:
os.listdir('../data')

['.DS_Store',
 'Merged_graphics.csv',
 'Merged_ai.csv',
 'output.json',
 'semantic_scholar_articles_20000.csv',
 'Merged_vision.csv',
 'Merged_nlp.csv',
 'output_compiled.json']

In [11]:
with open('../data/output_compiled.json','r') as f:
    citations_compiled = json.load(f)

In [22]:
semantic_df = pd.read_csv('../data/semantic_scholar_articles_20000.csv')

In [24]:
citations_compiled[0]

{'search_text': 'Transforming Auto-Encoders',
 'semantic_df_index': 0,
 'top_hits': [{'@score': '3',
   '@id': '3107620',
   'info': {'authors': {'author': [{'@pid': '10/3248',
       'text': 'Geoffrey E. Hinton'},
      {'@pid': '64/9381', 'text': 'Alex Krizhevsky'},
      {'@pid': '71/9736', 'text': 'Sida D. Wang'}]},
    'title': 'Transforming Auto-Encoders.',
    'venue': 'ICANN',
    'pages': '44-51',
    'year': '2011',
    'type': 'Conference and Workshop Papers',
    'key': 'conf/icann/HintonKW11',
    'doi': '10.1007/978-3-642-21735-7_6',
    'ee': 'https://doi.org/10.1007/978-3-642-21735-7_6',
    'url': 'https://dblp.org/rec/conf/icann/HintonKW11'},
   'url': 'URL#3107620',
   'references_crossref': [{'issue': '9',
     'key': '6_CR1',
     'doi-asserted-by': 'publisher',
     'first-page': '1',
     'DOI': '10.1371/journal.pcbi.1000495',
     'volume': '5',
     'author': 'P. Berkes',
     'year': '2009',
     'unstructured': 'Berkes, P., Turner, R.E., Sahani, M.: A structu

In [25]:
semantic_df.iloc[0]

abstract                    The artificial neural networks that are used t...
arxivId                                                                   NaN
authors                     [{'authorId': '1695689', 'name': 'Geoffrey E. ...
citationVelocity                                                          150
citations                   [{'arxivId': None, 'authors': [{'authorId': '1...
corpusId                                                              6138085
doi                                               10.1007/978-3-642-21735-7_6
influentialCitationCount                                                   52
paperId                              20f0357688876fa4662f806f985779dce6e24f3c
references                  [{'arxivId': None, 'authors': [{'authorId': '5...
title                                              Transforming Auto-Encoders
topics                      [{'topic': 'Computer vision', 'topicId': '5332...
url                         https://www.semanticscholar.org/pape

In [218]:
def authorComparison(s_authors,j_authors,year):
    score = []
    thres = 9 - (2.5*year)
    
    if len(s_authors)==0 and len(j_authors)==0:
        return 0 
    if len(s_authors)==len(j_authors):
        for a1,a2 in zip(s_authors,j_authors):
            score.append(needleman_wunsch_score(a1, a2, match=2, mismatch=-1, gap=-0.5, score_table=None))
        if sum(score)/len(score)>thres:
            return 1
    return 0
def getPlainString(string):
    return ''.join([s for s in string if not s.isnumeric()]).strip()

In [273]:
total_matches = []

for c1 in range(len(semantic_df)):
    i = semantic_df.iloc[c1]
    for c2 in range(len(citations_compiled[c1]['top_hits'])):
        j = citations_compiled[c1]['top_hits'][c2]
        
        try:
            s_doi = i['doi'].upper()
            j_doi = j['info']['doi'].upper()
        except:
            s_doi=-1
            j_doi=-2
        
        try:
            s_year = int(i['year'])
            j_year = int(j['info']['year'])
        except:
            s_year = -1
            j_year = -2
            
        try:
            s_authors = sorted([getPlainString(s['name']) for s in ast.literal_eval(i['authors'])],key = lambda x:x.split(" ")[-1])
            j_authors = sorted([getPlainString(s['text']) for s in j['info']['authors']['author']],key = lambda x:x.split(" ")[-1])
        except:
            try:
                j_authors = [j['info']['authors']['author']['text']]
            except:
                s_authors=[]
                j_authors = []

        if s_doi == j_doi or authorComparison(s_authors,j_authors,s_year == j_year)==1:
#             print(str(c1)+" "+str(c2)+" MATCH")
            total_matches.append((c1,c2))

        else:
            pass
#             print(str(c1)+" "+str(c2)+" NO MATCH")
    



In [392]:
total_matches_compiled = {}
t=0
while t<19413:
# for t in range(0,len(total_matches)):
#     print(total_matches[t])
    dc = [total_matches[t][1]]
    st = total_matches[t][0]
    for l in range(t+1,len(total_matches)):
#         print(total_matches[l])
        if total_matches[t][0] == total_matches[l][0]:
            dc.append(total_matches[l][1])

            continue
        else:
            total_matches_compiled[st]=dc
            t=l
            break
#     print(t)

In [433]:
total_matches_compiled_2 ={}
for i in total_matches_compiled:
    found1,found2,found3 = False,False,False
    for j in total_matches_compiled[i]:
        ts = (citations_compiled[i]['top_hits'][j])
        if 'references_crossref' in ts and 'references_opencitations' in ts:
            found1 = True
            f1 = j
        elif 'references_crossref' in ts:
            found2 = True
            f2 = j
        elif 'references_opencitations' in ts:
            found3 = True
            f3 = j
    if found1:
        ans = f1
    elif found2:
        ans= f2
    elif found3:
        ans = f3
    else:
        ans = 0
    total_matches_compiled_2[i]=ans

In [452]:
df_js = json.loads(semantic_df.to_json(orient='records'))

In [453]:
for i in total_matches_compiled_2:
    j = total_matches_compiled_2[i]
    df_js[i]['dblp_info'] = citations_compiled[i]['top_hits'][j]
    

In [455]:
f = open('semantic_joined_dblp.json','w+')
json.dump(df_js,f)
f.close()

In [457]:
df_js[0].keys()

dict_keys(['abstract', 'arxivId', 'authors', 'citationVelocity', 'citations', 'corpusId', 'doi', 'influentialCitationCount', 'paperId', 'references', 'title', 'topics', 'url', 'year', 'dblp_info'])

In [459]:
df_js[0]['dblp_info']

{'@score': '3',
 '@id': '3107620',
 'info': {'authors': {'author': [{'@pid': '10/3248',
     'text': 'Geoffrey E. Hinton'},
    {'@pid': '64/9381', 'text': 'Alex Krizhevsky'},
    {'@pid': '71/9736', 'text': 'Sida D. Wang'}]},
  'title': 'Transforming Auto-Encoders.',
  'venue': 'ICANN',
  'pages': '44-51',
  'year': '2011',
  'type': 'Conference and Workshop Papers',
  'key': 'conf/icann/HintonKW11',
  'doi': '10.1007/978-3-642-21735-7_6',
  'ee': 'https://doi.org/10.1007/978-3-642-21735-7_6',
  'url': 'https://dblp.org/rec/conf/icann/HintonKW11'},
 'url': 'URL#3107620',
 'references_crossref': [{'issue': '9',
   'key': '6_CR1',
   'doi-asserted-by': 'publisher',
   'first-page': '1',
   'DOI': '10.1371/journal.pcbi.1000495',
   'volume': '5',
   'author': 'P. Berkes',
   'year': '2009',
   'unstructured': 'Berkes, P., Turner, R.E., Sahani, M.: A structured model of video reproduces primary visual cortical organisation. PLoS Computational Biology\xa05(9), 1–16 (2009)',
   'journal-tit